# HDS 5230
## Week 6

High Performance Computing

In [1]:
conda install dask

Note: you may need to restart the kernel to use updated packages.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Derek\anaconda3

  added / updated specs:
    - dask


The following packages will be downloaded:


    package                    |            build
    ---------------------------|-----------------
    conda-4.11.0               |   py39haa95532_0        14.4 MB
    ------------------------------------------------------------
                                           Total:        14.4 MB

The following packages will be UPDATED:

  conda                               4.10.3-py39haa95532_0 --> 4.11.0-py39haa95532_0




conda-4.11.0         | 14.4 MB   |            |   0% 
conda-4.11.0         | 14.4 MB   | 1          |   2% 
conda-4.11.0         | 14.4 MB   | ####5      |  45% 
conda-4.11.0         | 14.4 MB   | ########## | 100% 
conda-4.11.0         | 14.4 MB   | ########## | 100% 
Preparing transaction: ...working... done
V

In [1]:
import os
import pandas as pd
import numpy as np
from dask.distributed import Client
from multiprocessing import cpu_count
import dask.dataframe as dd

In [2]:
ncores = cpu_count()
client = Client()

ncores

24

In [3]:
filename = os.path.join('timeseries.csv')

dtype={'cases': 'float64',
       'county': 'object',
       'deaths': 'float64',
       'recovered': 'float64',
       'state': 'object'}

data_pd = pd.read_csv(filename, dtype=dtype)

data = dd.from_pandas(data_pd, npartitions = ncores)

In [4]:
data.head()

,locationID,slug,name,level,city,county,state,country,lat,long,...,deaths,recovered,active,tested,hospitalized,hospitalized_current,discharged,icu,icu_current,date
0,iso1:ad,ad,Andorra,country,NaN,NaN,NaN,Andorra,42.55,1.58,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-22
1,iso1:ad,ad,Andorra,country,NaN,NaN,NaN,Andorra,42.55,1.58,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-23
2,iso1:ad,ad,Andorra,country,NaN,NaN,NaN,Andorra,42.55,1.58,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-24
3,iso1:ad,ad,Andorra,country,NaN,NaN,NaN,Andorra,42.55,1.58,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-25
4,iso1:ad,ad,Andorra,country,NaN,NaN,NaN,Andorra,42.55,1.58,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-26


In [11]:
data.columns

Index(['locationID', 'slug', 'name', 'level', 'city', 'county', 'state',
       'country', 'lat', 'long', 'population', 'aggregate', 'tz', 'cases',
       'deaths', 'recovered', 'active', 'tested', 'hospitalized',
       'hospitalized_current', 'discharged', 'icu', 'icu_current', 'date'],
      dtype='object')

In [16]:
grouped = data[(data['date'] >= '2020-01-01') & (data['date'] <= '2020-02-28') & (data['country'] == 'United States')].groupby('state')
grouped

In [19]:
def chunk(g):
    return g['deaths'].sum(), g['population'].mean()

def agg(deaths, pops):
    return deaths.sum(), pops.mean()

def finalize(deaths, pops):
    return deaths / pops

mort = dd.Aggregation('mort', chunk, agg, finalize=finalize)

grouped.agg(mort).compute


ValueError: Metadata inference failed in `_groupby_apply_funcs`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
IndexError('Column(s) active already selected')

Traceback:
---------
  File "C:\Users\Derek\anaconda3\lib\site-packages\dask\dataframe\utils.py", line 176, in raise_on_meta_error
    yield
  File "C:\Users\Derek\anaconda3\lib\site-packages\dask\dataframe\core.py", line 5787, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "C:\Users\Derek\anaconda3\lib\site-packages\dask\dataframe\groupby.py", line 910, in _groupby_apply_funcs
    r = func(grouped, **func_kwargs)
  File "C:\Users\Derek\anaconda3\lib\site-packages\dask\dataframe\groupby.py", line 956, in _apply_func_to_column
    return func(

In [13]:
grouped['deaths'].sum().compute()

state
Alabama                          0.0
Alaska                           0.0
Arkansas                         0.0
American Samoa                   0.0
Arizona                          0.0
California                      35.0
Colorado                         0.0
Connecticut                      0.0
Delaware                         0.0
Florida                          0.0
Georgia                          0.0
Washington, D.C.                 0.0
Guam                             0.0
Hawaii                           0.0
Iowa                             0.0
Idaho                            0.0
Illinois                         0.0
Indiana                          0.0
Kansas                           0.0
Kentucky                         0.0
Louisiana                        0.0
Maine                            0.0
Maryland                         0.0
Massachusetts                    0.0
Michigan                         0.0
Minnesota                        0.0
Missouri                        